In [1]:
import requests
import functools
import re
from itertools import chain
from bs4 import BeautifulSoup
from urllib.parse import quote_plus

In [2]:
def get(url, query):
    quoted_query = quote_plus(query)
    underscored_query = query.replace(' ', '_')
    full_url = '{}?q={}'.format(url, quoted_query)
    r = requests.get(full_url)
    status = r.status_code
    if status is 200:
        soup = BeautifulSoup(r.text, 'html.parser')
        return soup
    else:
        print('Error code: {}; URL: {}'.format(status, full_url))

In [3]:
def yahoo(query): return yahooParser(get('https://search.yahoo.com/search;', query))
def yahooParser(soup):
    matchURLsThatArentYahoosIP = re.compile("(?=^http.)(?!^http:\/\/98)")
    selectResultsTag           = lambda soup : soup.select("ol.searchCenterMiddle")[0]
    removeNewsAndSpecialLinks  = lambda soup : soup.select("div.wrapstar")+soup.select("div.algo")
    selectSearchResults        = lambda soup : removeNewsAndSpecialLinks(selectResultsTag(soup))
    removeInvalidURLs          = lambda url  : matchURLsThatArentYahoosIP.search(url)
    selectLinkTag              = lambda div  : div.find_all('a', href=True)
    selectLinkURL              = lambda link : link["href"]
    flatmap                    = lambda fun, lst : list(chain.from_iterable(map(fun,lst)))
    
    return list(filter(removeInvalidURLs,
                map(selectLinkURL,
                flatmap(selectLinkTag,
                        selectSearchResults(soup)))))

In [12]:
def google(query): return googleParser(get('https://www.google.com/search', query))
def googleParser(soup):
    matchNonSpecialResults = re.compile('^(?!News for|Images for).*$')
    selectResultTags       = lambda soup : soup.select('div#ires div.g h3.r a')
    removeSpecialResults   = lambda tags : [tag for tag in tags if matchNonSpecialResults.search(tag.text)]
    selectLinkURL          = lambda link : link['href']
    trimURL                = lambda url  : url.strip('/url?q=').split('&')[0]
    
    return list(map(trimURL,
                map(selectLinkURL,
                    removeSpecialResults(selectResultTags(soup)))))

In [6]:
def bing(query): return bingParser(get('http://www.bing.com/search', query))
def bingParser(soup): 
    return [t.h2.a['href'] for t in soup.find_all('li', 'b_algo')]

In [7]:
yahoo("test")

['http://www.speedtest.net/',
 'http://www.thefreedictionary.com/test',
 'http://www.test.com/',
 'https://www.speakeasy.net/speedtest/',
 'http://www.dictionary.com/browse/test',
 'https://en.wikipedia.org/wiki/Test_(student_assessment)',
 'http://www.merriam-webster.com/dictionary/test',
 'https://en.wikipedia.org/wiki/Test',
 'http://www.tests.com/',
 'http://www.queendom.com/tests/index.htm']

In [11]:
google("test")

['https://www.test.com/',
 'http://www.speedtest.net/',
 'https://en.wikipedia.org/wiki/Test',
 'https://www.speakeasy.net/speedtest/',
 'http://speedtest.xfinity.com/',
 'http://thehill.com/blogs/ballot-box/presidential-races/301181-trump-calls-for-drug-test-before-debate',
 'http://www.springer.com/statistics/journal/11749',
 'https://www.16personalities.com/free-personality-test',
 'http://www.reuters.com/article/us-korea-north-usa-idUSKBN12F0S2%3FfeedType%3DRSS%26feedName%3DtopNews%26rpc%3D69']

In [9]:
bing("test")

['http://www.speedtest.net/',
 'http://www.thefreedictionary.com/test',
 'http://www.test.com/',
 'https://www.speakeasy.net/speedtest/',
 'http://www.dictionary.com/browse/test',
 'https://en.wikipedia.org/wiki/Test_(student_assessment)',
 'https://en.wikipedia.org/wiki/Test',
 'http://www.merriam-webster.com/dictionary/test',
 'http://www.tests.com/']